In [2]:

import scipy.io
from sklearn import svm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mat4py import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
import time 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import KFold
from keras.utils import to_categorical
import sklearn.metrics as metrics

In [3]:
maindir = 'preprocessedData'

In [4]:
eegs= []
labels = []

In [5]:
directory_contents = os.listdir(maindir)
for mydir in directory_contents:
    print(mydir)
    if(mydir != '.DS_Store'):
        mydir = os.path.join(maindir, mydir)
        for fname in os.listdir(mydir):
            #print(fname,)
            fpath = os.path.join(mydir, fname)
            currentEegData = pd.read_csv(fpath, header = None)
            currentEeg = currentEegData.to_numpy()
            eegs.append(currentEeg)
            if(fname[3:9] == 'target'):
                #print('it is target')
                currentLabel = 1
                labels.append(currentLabel)
            elif(fname[3:13] == 'non-target'):
                #print('it is target')
                currentLabel = -1
                labels.append(currentLabel)

S4_targets
S5_non-targets
.DS_Store
S5_targets
S3_non-targets
S1_non-targets
S3_targets
S2_targets
S4_non-targets
S1_targets
S2_non-targets


In [6]:
eegsNp = np.array(eegs)
eegsNp.shape

(6000, 201, 8)

In [7]:
labelsNp = np.array(labels)
print(labelsNp.shape)

(6000,)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(eegsNp, labelsNp, test_size=0.2, random_state=4)

In [9]:
x_train.shape

(4800, 201, 8)

In [10]:
y_train.shape

(4800,)

In [11]:
x_train_average = np.average(x_train, axis=2)
print(x_train_average.shape)

(4800, 201)


In [12]:
x_test_average = np.average(x_test, axis=2)
print(x_test_average.shape)

(1200, 201)


# LDA


In [13]:
lda = LinearDiscriminantAnalysis()

In [14]:
start = time.time()
LDA_avg = lda.fit(x_train_average, y_train).transform(x_train_average)
stop = time.time()
print("time:", stop-start)

time: 0.23710203170776367


In [15]:
start = time.time()
prediction= lda.predict(x_test_average)
stop = time.time()
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))
print("time:", stop - start)

accuracy: 0.9891666666666666
recall: 0.9103448275862069
precision: 1.0
roc-auc: 0.9551724137931035
f-measure: 0.9530685920577617
time: 0.00146484375


# KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
kNN_Manhattan = KNeighborsClassifier(n_neighbors=3, weights='distance', p = 1)

In [17]:
start = time.time()
kNN_Manh_avg = kNN_Manhattan.fit(x_train_average, y_train)
stop = time.time()
print("training time:", stop-start)

training time: 0.05698418617248535


In [18]:
start = time.time()
prediction= kNN_Manhattan.predict(x_test_average)
stop = time.time() 
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))
print("time:", stop - start)

accuracy: 0.9966666666666667
recall: 0.9724137931034482
precision: 1.0
roc-auc: 0.9862068965517241
f-measure: 0.9860139860139859
time: 0.5218160152435303


# SVM

In [19]:
from sklearn import svm
svmClf = svm.SVC(probability=True)
start = time.time()
svmClf.fit(x_train_average, y_train)
stop = time.time()
print(stop-start)

1.344454050064087


In [20]:
start = time.time()
prediction = svmClf.predict(x_test_average)
stop = time.time() 
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))
print("time:", stop - start)

accuracy: 0.9991666666666666
recall: 0.993103448275862
precision: 1.0
roc-auc: 0.996551724137931
f-measure: 0.9965397923875432
time: 0.04038095474243164


# Ensemble

## lda-knn

In [21]:
from sklearn.ensemble import VotingClassifier

lda_knn_maxvote = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda)], voting='hard')
start = time.time() 
lda_knn_maxvote.fit(x_train_average,y_train)
stop = time.time()                                    
print("time", stop-start)
prediction = lda_knn_maxvote.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 0.24949097633361816
accuracy: 0.9858333333333333
recall: 0.8827586206896552
precision: 1.0
roc-auc: 0.9413793103448276
f-measure: 0.9377289377289377


In [22]:
lda_knn_argmax = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda)], voting='soft')
start = time.time() 
lda_knn_argmax.fit(x_train_average,y_train)
stop = time.time()                                    
print("time", stop-start)
prediction = lda_knn_argmax.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 0.21390604972839355
accuracy: 0.9966666666666667
recall: 0.9724137931034482
precision: 1.0
roc-auc: 0.9862068965517241
f-measure: 0.9860139860139859


# lda-knn-svm

In [23]:
lda_knn_svm = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda), ('svm', svmClf)], voting='soft')
start = time.time()
lda_knn_svm.fit(x_train_average,y_train)
stop = time.time()                                    
print("time", stop-start)
prediction = lda_knn_svm.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 1.495192050933838
accuracy: 0.9991666666666666
recall: 0.993103448275862
precision: 1.0
roc-auc: 0.996551724137931
f-measure: 0.9965397923875432


In [24]:
lda_knn_svm_maxvote = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda), ('svm', svmClf)], voting='hard')
start = time.time()
lda_knn_svm_maxvote.fit(x_train_average,y_train)
stop = time.time()                                    
print("time", stop-start)
start = time.time()
prediction = lda_knn_svm_maxvote.predict(x_test_average)
stop = time.time()                                    
print("prediction time", stop-start)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 1.511268138885498
prediction time 0.5625121593475342
accuracy: 0.9991666666666666
recall: 0.993103448275862
precision: 1.0
roc-auc: 0.996551724137931
f-measure: 0.9965397923875432


# Ensemble voting with Grid Search

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
knn_params = {'n_neighbors':[3, 5, 7, 9, 11],
              'weights':['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

knn_gs = GridSearchCV(KNeighborsClassifier(), knn_params, verbose=1, cv = 3, n_jobs = -1)
knn_gs_results = knn_gs.fit(x_train_average, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   13.1s finished


In [27]:
knn_gs_results.best_params_

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

In [35]:
from sklearn.preprocessing import StandardScaler 
ss = StandardScaler()
x_train_scaled = ss.fit_transform(x_train_average)
x_test_scaled = ss.transform(x_test_average)


In [40]:
weights_params = {'weights':[[0.33, 0.33, 0.33],
                             [0.5, 0.25, 0.25],[0.25, 0.5, 0.25],[0.25, 0.25, 0.5], 
                             [0.4, 0.3, 0.3], [0.3, 0.4, 0.3], [0.3, 0.3, 0.4]]}


lda_knn_svm = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda), ('svm', svmClf)], voting='soft', verbose=1)

ensemble_gs = GridSearchCV(lda_knn_svm, weights_params)

start = time.time()
ensemble_gs.fit(x_train_average,y_train)
stop = time.time()                                    


[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.2s
[Votin

In [50]:
ensemble_gs.cv_results_

{'mean_fit_time': array([1.10876846, 1.12874141, 1.12474523, 1.10555124, 1.09188075,
        1.09973731, 1.10568857]),
 'std_fit_time': array([0.03001134, 0.00892538, 0.03734621, 0.02235351, 0.0134729 ,
        0.02191   , 0.02800304]),
 'mean_score_time': array([0.32913427, 0.3368165 , 0.32857819, 0.33277054, 0.32427588,
        0.32669878, 0.3308105 ]),
 'std_score_time': array([0.01333946, 0.01127541, 0.01213487, 0.0097417 , 0.01301679,
        0.01468292, 0.00936529]),
 'param_weights': masked_array(data=[list([0.33, 0.33, 0.33]), list([0.5, 0.25, 0.25]),
                    list([0.25, 0.5, 0.25]), list([0.25, 0.25, 0.5]),
                    list([0.4, 0.3, 0.3]), list([0.3, 0.4, 0.3]),
                    list([0.3, 0.3, 0.4])],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'weights': [0.33, 0.33, 0.33]},
  {'weights': [0.5, 0.25, 0.25]},
  {'weights': [0.25, 0.5, 0.25]},
  {'weights': [0.25

In [51]:
means = ensemble_gs.cv_results_['mean_test_score']
stds = ensemble_gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, ensemble_gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"%(mean, std * 2, params))
    print()

1.000 (+/-0.001) for {'weights': [0.33, 0.33, 0.33]}

0.999 (+/-0.002) for {'weights': [0.5, 0.25, 0.25]}

0.996 (+/-0.002) for {'weights': [0.25, 0.5, 0.25]}

1.000 (+/-0.000) for {'weights': [0.25, 0.25, 0.5]}

1.000 (+/-0.001) for {'weights': [0.4, 0.3, 0.3]}

0.999 (+/-0.001) for {'weights': [0.3, 0.4, 0.3]}

1.000 (+/-0.000) for {'weights': [0.3, 0.3, 0.4]}



# Ensemble voting with Random Search


In [28]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
weights_params = {'weights':[[0.33, 0.33, 0.33],
                             [0.5, 0.25, 0.25],[0.25, 0.5, 0.25],[0.25, 0.25, 0.5], 
                             [0.4, 0.3, 0.3], [0.3, 0.4, 0.3], [0.3, 0.3, 0.4]]}

In [46]:
import random

In [47]:
weights_rand= {'weights':[[random.uniform(0.1, 0.9), random.uniform(0.1, 0.9), random.uniform(0.1, 0.9)],
                         [random.uniform(0.1, 0.9), random.uniform(0.1, 0.9), random.uniform(0.1, 0.9)], 
                          [random.uniform(0.1, 0.9), random.uniform(0.1, 0.9), random.uniform(0.1, 0.9)]]}

In [48]:


lda_knn_svm = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda), ('svm', svmClf)], voting='soft', verbose=1)
#ensemble_gs = GridSearchCV(lda_knn_svm, weights_params)
random_voter = RandomizedSearchCV(lda_knn_svm, weights_rand)

In [49]:
random_voter.fit(x_train_average, y_train)

/Users/apple/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.2s
[Voting] ...................... (3 of 3) Processing svm, total=   1.0s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Voting] ...................... (3 of 3) Processing svm, total=   0.9s
[Voting] ...................... (1 of 3) Processing knn, total=   0.0s
[Voting] ...................... (2 of 3) Processing lda, total=   0.1s
[Votin

RandomizedSearchCV(estimator=VotingClassifier(estimators=[('knn',
                                                           KNeighborsClassifier(n_neighbors=3,
                                                                                p=1,
                                                                                weights='distance')),
                                                          ('lda',
                                                           LinearDiscriminantAnalysis()),
                                                          ('svm',
                                                           SVC(probability=True))],
                                              verbose=1, voting='soft'),
                   param_distributions={'weights': [[0.6328520719512413,
                                                     0.6663306545655543,
                                                     0.3599861621869408],
                                                    [

In [50]:
random_voter.best_params_

{'weights': [0.20987949099695957, 0.14023819025684878, 0.812441268062074]}

In [51]:
random_voter.best_score_

1.0

In [52]:
random_voter.predict(x_train_average)

array([-1, -1, -1, ..., -1, -1, -1])

In [53]:
lda_knn_svm_w1 = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda), ('svm', svmClf)], voting='soft', weights =[0.2098, 0.1402, 0.8124])
start = time.time()
lda_knn_svm_w1.fit(x_train_average,y_train)
stop = time.time()                                    
print("time", stop-start)
prediction = lda_knn_svm_w1.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 1.5380220413208008
accuracy: 0.9991666666666666
recall: 0.993103448275862
precision: 1.0
roc-auc: 0.996551724137931
f-measure: 0.9965397923875432


In [54]:
lda_knn_svm_gs = VotingClassifier(estimators=[('knn', kNN_Manhattan), ('lda', lda), ('svm', svmClf)], voting='soft', weights =[0.2098, 0.1402, 0.8124])
start = time.time()
lda_knn_svm_gs.fit(x_train_average,y_train)
stop = time.time()                                    
print("time", stop-start)
prediction = lda_knn_svm_gs.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 1.6066091060638428
accuracy: 0.9991666666666666
recall: 0.993103448275862
precision: 1.0
roc-auc: 0.996551724137931
f-measure: 0.9965397923875432


# Boosting

In [25]:
import xgboost as xgb

In [26]:
xgboost=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
start = time.time()
xgboost.fit(x_train_average, y_train)
stop = time.time()
print("time", stop - start)
prediction = xgboost.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

[19:59:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/apple/.pyenv/versions/3.7.3/lib/python3.7/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


time 1.882566213607788
accuracy: 0.9941666666666666
recall: 0.9517241379310345
precision: 1.0
roc-auc: 0.9758620689655173
f-measure: 0.9752650176678445


In [122]:
from sklearn.ensemble import GradientBoostingClassifier
gradboost= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
start = time.time()
gradboost.fit(x_train_average, y_train)
stop = time.time()
print("time", stop - start)
prediction = gradboost.predict(x_test_average)
print("accuracy:",metrics.accuracy_score(y_test, prediction))
print("recall:",metrics.recall_score(y_test, prediction))
print("precision:",metrics.precision_score(y_test, prediction))
print("roc-auc:",metrics.roc_auc_score(y_test, prediction))
print("f-measure:",metrics.f1_score(y_test, prediction))

time 6.497662305831909
accuracy: 0.9683333333333334
recall: 0.7379310344827587
precision: 1.0
roc-auc: 0.8689655172413793
f-measure: 0.8492063492063493


# CNN 

In [60]:
import tensorflow as tf

from tensorflow import keras
tf.keras.backend.clear_session()  # For easy reset of notebook state.

from tensorflow.keras import layers

input_layer = keras.Input(shape = (201,8,1), name='main_input')
x     = layers.Conv2D(16, 8, padding='same', activation='relu')(input_layer)
x     = layers.Conv2D(32, 6, padding='same', activation='relu')(x)
x     = layers.Conv2D(8, 4, padding='same', activation='relu')(x)
x     = layers.Conv2D(4, 2, padding='same', activation='relu')(x)
x     = layers.GlobalAveragePooling2D()(x)
x     = layers.Dense(8)(x)
x     = layers.Dense(64)(x)
output = layers.Dense(2, activation='softmax')(x)

cnn = keras.Model(inputs=input_layer, outputs=output)

cnn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 201, 8, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 201, 8, 16)        1040      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 201, 8, 32)        18464     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 201, 8, 8)         4104      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 201, 8, 4)         132       
_________________________________________________________________
global_average_pooling2d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 8)                 40    

In [114]:
#compiling the model
opt = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=True)  # default params
cnn.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [115]:
start = time.time()
history = cnn.fit(x_train, y_train, batch_size=1, epochs=20)
stop = time.time()
print(stop - start)

Epoch 1/20
4800/4800 [==============================] - 61s 13ms/step - loss: 0.6931 - accuracy: 0.0590
Epoch 2/20
4800/4800 [==============================] - 63s 13ms/step - loss: 0.6931 - accuracy: 0.0640
Epoch 3/20
4800/4800 [==============================] - 63s 13ms/step - loss: 0.6931 - accuracy: 0.0600
Epoch 4/20
4800/4800 [==============================] - 63s 13ms/step - loss: 0.6931 - accuracy: 0.0575
Epoch 5/20
4800/4800 [==============================] - 64s 13ms/step - loss: 0.6931 - accuracy: 0.0556
Epoch 6/20
4800/4800 [==============================] - 59s 12ms/step - loss: 0.6931 - accuracy: 0.0615
Epoch 7/20
4800/4800 [==============================] - 63s 13ms/step - loss: 0.6931 - accuracy: 0.0654
Epoch 8/20
4800/4800 [==============================] - 62s 13ms/step - loss: 0.6931 - accuracy: 0.0656
Epoch 9/20
4800/4800 [==============================] - 61s 13ms/step - loss: 0.6931 - accuracy: 0.0623
Epoch 10/20
4800/4800 [==============================] - 63s 13m

In [121]:
prediction = cnn.predict(x_test)
prediction

array([[0.49999818, 0.5000018 ],
       [0.49999818, 0.5000018 ],
       [0.49999818, 0.5000018 ],
       ...,
       [0.49999818, 0.5000018 ],
       [0.49999818, 0.5000018 ],
       [0.49999818, 0.5000018 ]], dtype=float32)